# Annotating Human Proteome query results containing site and region features onto posID level data
3.22.2022 
@mfpfox

### help links
* Sequence annotation (Features) <https://www.uniprot.org/help/sequence_annotation>
* api query <https://www.uniprot.org/help/api_queries>

---

# **columns parsed/formatted by downloadUniprotDomains.py:**
## SITES
1. SubstrateBindingSite
2. ActiveSite
3. MetalBindSite
4. ModifiedResidue
5. Lipidation
6. Glycosylation
7. Mutagenesis
8. DisulfideBond

FORMAT: seperator "; " disulfide sep '..'

## REGIONS
0. topological domain (not included in saved output from script)
1. Domain
2. NucleotideBindRegion
3. DNAbindRegion
4. ZincFinger
5. Motif
6. DisorderedRegion
7. BetaStrand
8. Helix
9. Turn

FORMAT: regions '-' and '; ' 

## proximal defined by not at site and not at region upper or lower bound position, but within +5 of high and -5 positions of region bounds and +- 5 of site positions

In [1]:
import sys
import os
sys.path.append("/Users/mariapalafox/Desktop/TOOLBOXPY")
from all_funx import *
from IPython.display import display, HTML
from IPython.display import Image
display(HTML("<style>.container {width:90% !important;}</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_colwidth', 1000)
pd.options.display.max_seq_items = 2000
os.chdir("/Users/mariapalafox/Desktop/BRIDGE/P4_UniProt_Query/isPositionROI/")

### Filtered downloadUniprotDomains.py output file that was QC'd for ukb sequence match

* dir: `/Users/mariapalafox/Desktop/BRIDGE/P4_UniProt_Query/isPositionROI/Uniprot_Domains/20220212/`

* unfiltered file made by downloadUniprotDomains.py:`features_summary_QCd_sequences.csv`

### Proceeding to annotated ukb sites and regions (exact match OR proximal match) for positions of sequences that have identical UKB 2/12/22 & UKB CpDAA ref sequences

* filtered ukb annotation data made in above cells:`features_summary_QCd_IdenticalSeq_18231proteins.csv`

In [78]:
print("Number of posID proteins found in ukb annotation df \n",
      4535-4497, "\n", "n=4497 now, was 4535 pre merge\n",
      "Percent protein IDs lost:" ,(38/4535)*100)
print('Total of ', 18827-18708, ' CpDAA posIDs lost from merge')
print("had 18827, now have 18708 cpdaa")

Number of posID proteins found in ukb annotation df 
 38 
 n=4497 now, was 4535 pre merge
 Percent protein IDs lost: 0.8379272326350606
Total of  119  CpDAA posIDs lost from merge
had 18827, now have 18708 cpdaa


In [20]:
# import ukb annotation file drop sequence col reorder cols

dirname = "Uniprot_Domains/20220212/"
df = pd.read_csv(dirname+"features_summary_QCd_sequences.csv")
#--------------------------------------------
# save protein level file
#  # protein -level merge later
# 'Protein', 'Gene', 'Length','SubunitStructure', '3D', 'PDB', 'InteractsWith', 'PTM', 
proteinlevel = df[['UKBID', 
         'identicalToRefSequence',
           'query.minus.REF.difference', 
             'QC.posID.difference',
                 'Protein', 'Gene', 'Length','SubunitStructure',
               '3D', 'PDB', 'InteractsWith', 'PTM']].copy()
proteinlevel.to_csv("features_summary_QCd_02122022_proteinLevelColumns.csv", index=False)
#--------------------------------------------
# simplifly  'Sequence', 
df = df[['UKBID', 
         'identicalToRefSequence',
           'query.minus.REF.difference', 
             'QC.posID.difference',
                 'Protein', 'Gene', 'Length','SubunitStructure',
               '3D', 'PDB', 'InteractsWith', 'PTM', 
             'SubstrateBindingSite',
             'ActiveSite', 
             'MetalBindSite', 
             'ModifiedResidue',
             'Lipidation',
             'Glycosylation', 
             'Mutagenesis', 
                'DisulfideBond',
    'NucleotideBindRegion', 'DNAbindRegion',
   'ZincFinger', 'Motif', 
         'DisorderedRegion', 'Domain', 
   'BetaStrand', 'Helix', 
         'Turn'
         ]].copy()

In [21]:
#qc
print(checkColumnValues(df, 'identicalToRefSequence'))

   identicalToRefSequence  Count
0                    True  18231
1                   False    185


In [10]:
# save dropped protein annotation rows 
falsedf =  df[~df['identicalToRefSequence']].copy()
print(falsedf.shape)

falsedf.to_csv("features_summary_QCd_NotIdenticalSequence_185proteins.csv", index=False)
# saving falsedf , sequences dropped because they differed from the reference proteome

(185, 29)


In [23]:
# drop rows with different sequences between query and ref for cpdaa
df =  df[df['identicalToRefSequence']].copy()
print(df.shape)
df.head(1)

(18231, 29)


,UKBID,identicalToRefSequence,query.minus.REF.difference,QC.posID.difference,Protein,Gene,Length,SubunitStructure,3D,PDB,InteractsWith,PTM,SubstrateBindingSite,ActiveSite,MetalBindSite,ModifiedResidue,Lipidation,Glycosylation,Mutagenesis,DisulfideBond,NucleotideBindRegion,DNAbindRegion,ZincFinger,Motif,DisorderedRegion,Domain,BetaStrand,Helix,Turn
0,Q9GZX5,True,0,NaN,Zinc finger protein 350 (KRAB zinc finger protein ZFQR) (Zinc finger and BRCA1-interacting protein with a KRAB domain 1) (Zinc finger protein ZBRK1),ZNF350,532,"SUBUNIT: Interacts with BRCA1. Interacts with RNF11. {ECO:0000269|PubMed:11090615, ECO:0000269|PubMed:14755250}.",NaN,NaN,P38398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206-228; 234-256; 262-284; 290-312; 318-340; 346-368; 374-396; 402-424,NaN,427-465,8-79,NaN,NaN,NaN


In [29]:
# saving
df.to_csv("features_summary_QCd_IdenticalSeq_18231proteins.csv", index=False)

# isposition_atSite_inDomain.py functions for SITE and REGION ukb annotations

##### UPDATE THOUGHTS isposition_atSite_inDomain.py :
- add section of code that checks if query result sequence is identical to ref of cpdaa and category positions
    - if sequence is not identical, are positions of cpdaa and category unchanged?
    - if sequence is same length but different, qc that site modifications match reference aa type at position
- **add substrings corresponding to zinc fingers, domains, other regions that may be of interest for aa abundance analysis**

### tested functions in markdown, final functions in script


```python
#------------------------------code for script------------------------
#-------variables--------------

# 2 ARGS for filenames,all positions MendelCpD: codonMaxMeanScored_mergedDescribeProt_1231_MendelCpD.csv
POSINFILE = "REF_posID_level_18827cpdaa_4535UKBIDs.csv"
# import ukb annotation reference with only identical ukb sequences
UKBINFILE = "features_summary_QCd_IdenticalSeq_18231proteins.csv"
# 2 ARGS name of column with UKBIDs for both dataframes
IDname = "UKBID"
posIDname = "posID"
saveSite = "CpDAA_UKBfeatures_Sites_mergedAnnotations.csv"
saveRegion = "CpDAA_UKBfeatures_Region_mergedAnnotations.csv"
#-------what code does--------------
# imports and merges on position index ref ukbIDs, so only looking at proteins with annotations from ukb
# adds an integer pos column from split posID
# splits merged df into site and region dfs, drops columns other than posID, pos, site|regoin annotions
# *makes new header column with exact and proximal colnames added for site and region dfs

positions = pd.read_csv(POSINFILE)
ukb = pd.read_csv(UKBINFILE)
merged1 = pd.merge(positions, ukb, on=[IDname], how="inner") # MERGES UKB ANNOTATION ONTO POSITION LEVEL DF
merged1= getAAposition(merged1, 'posID', 'pos') # ADDS POS INTEGER COL BY SPLITTING POSID

# *MAKE HEADER COLUMNS FOR MERGED DF'S SPECIFIC TO SITE AND REGION COLUMNS
basicHead = ['posID', 'pos']
sitesCol= ['SubstrateBindingSite', 'ActiveSite', 'MetalBindSite',
       'ModifiedResidue', 'Lipidation', 'Glycosylation', 'Mutagenesis',
       'DisulfideBond'] # SITE
sitesDF = merged1[basicHead + sitesCol].copy()
sitesDF.to_csv(saveSite, index=False) # save site and region df's
sitesAddCol = [] # make exact and proximal header additions SITES
for coli in sitesCol:
    exx = coli + '_Exact'
    prx = coli + '_Proximal5'
    sitesAddCol.append(coli)
    sitesAddCol.append(exx)
    sitesAddCol.append(prx)
regionCol = [
        'NucleotideBindRegion', 'DNAbindRegion', 'ZincFinger',
       'Motif', 'DisorderedRegion', 'Domain', 'BetaStrand', 'Helix', 
        'Turn'] # REGION
regionDF = merged1[basicHead + regionCol].copy()
regionDF.to_csv(saveRegion, index=False) # save site and region df's
regAddCol = [] # make exact and proximal header additions REGIONS
for coli in regionCol:
    exx = coli + '_Exact'
    prx = coli + '_Proximal5'
    regAddCol.append(coli)
    regAddCol.append(exx)
    regAddCol.append(prx)
# NEW HEADERS
newheaderSites = basicHead + sitesAddCol
newheaderRegion = basicHead + regAddCol

# Function call outfile naming for site and region
outSite = saveSite.replace("_mergedAnnotations.csv", "_ANNOTATED.csv")
outRegion = saveRegion.replace("_mergedAnnotations.csv", "_ANNOTATED.csv")

addSiteAnnotations(saveSite, newheaderSites, outSite, sitesCol)
addRegionAnnotations(saveRegion, newheaderRegion, outRegion, regionCol)
```

```python
def addSiteAnnotations(infile, head, outfile, siteCol):
    f = open(infile)
    df = list(csv.reader(f))
    # df = df[0:2]
    with open(outfile, 'w') as out:
            csvWriter = csv.writer(out)
            csvWriter.writerow(head)
    # loop over rows skip header
    for row in df[1:]:
        #print(row)
        newrow = []
        newrow.append(row[0]) # posID
        pos = int(row[1])
        newrow.append(pos)
        annoRow = row[2:] # select site annotation columns, skip posID and pos
        for i, v in enumerate(annoRow):
            flounder = False # FLAG EXACT
            flounder2 = False # FLAG PROXIMAL
            siteType = siteCol[i]
            # nan annotation value case
            if (len(v) < 1) | (v == ''): 
                newrow.append(v) 
                newrow.append(flounder) # exact 
                newrow.append(flounder2) # proximal
            # not nan annotation value case
            if len(v) >= 1: 
                if siteType != 'DisulfideBond':
                    vsplit = v.split(";")
                    # EXACT
                    for exac in vsplit:
                        try:
                            posCheck = int(exac.strip(" "))
                        except:
                            print("error posCheck:", posCheck)
                            posCheck = -100
                        # check exact match 
                        if pos == posCheck:
                            flounder = True
                            break
                    # PROXIMAL
                    for prox in vsplit:
                        try:
                            posCheck = int(prox.strip(" "))
                        except:
                            print("error posCheck:", posCheck)
                            posCheck = -100
                        # check proximal match
                        if (pos >= posCheck-5 and pos < posCheck) | (pos <= posCheck+5 and pos > posCheck):
                            flounder2 = True
                            break
                    newrow.append(v)
                    newrow.append(flounder)
                    newrow.append(flounder2)
                # disulfide case          
                if siteType == 'DisulfideBond':
                    vsplit = v.split(";")
                    for exac in vsplit:
                        posCheck = exac.strip(" ")
                        posCheck = posCheck.split("..")
                        for disul in posCheck:
                            try:
                                di = int(disul)
                            except: 
                                print("error disul:", disul)
                                di = -100
                            # check exact match
                            if pos == di:
                                flounder = True
                                break
                    for prox in vsplit:
                        posCheck = prox.strip(" ")
                        posCheck = posCheck.split("..")
                        for pp in posCheck:
                            try:
                                pc = int(pp)
                            except:
                                print("error pp:", pp)
                                pc = -100
                            # check proximal match
                            if (pos >= pc-5 and pos < pc) | (pos <= pc+5 and pos > pc):
                                flounder2 = True
                                break
                    newrow.append(v)
                    newrow.append(flounder)
                    newrow.append(flounder2)
        with open(outfile, 'a') as out:
            csvWriter = csv.writer(out)
            csvWriter.writerow(newrow)
#addSiteAnnotations(saveSite, newheaderSites, outSite, sitesCol)
```

```python
def addRegionAnnotations(infile, head, outfile, siteCol):
    f = open(infile)
    df = list(csv.reader(f))
    #df = df[0:2]
    with open(outfile, 'w') as out:
            csvWriter = csv.writer(out)
            csvWriter.writerow(head)
    # loop over rows skip header
    for row in df[1:]:
        newrow = []
        newrow.append(row[0]) # posID
        pos = int(row[1]) # aa position to check in domain range
        newrow.append(pos)
        annoRow = row[2:] # select region annotation columns, skip posID and pos cols
        for i, v in enumerate(annoRow):
            flounder = False # FLAG EXACT
            flounder2 = False # FLAG PROXIMAL
            siteType = siteCol[i]
            # nan annotation value case, ADD EMPTY STRING TO FILTER
            if (len(v) < 1) | (v == ''): 
                newrow.append(v) 
                newrow.append(flounder) # exact 
                newrow.append(flounder2) # proximal
            # not nan annotation value case
            if len(v) >= 1:     
                vsplit = v.split(";")
                # EXACT
                for exac in vsplit:
                    posCheck = exac.strip(" ")
                    posCheck = posCheck.split("-")
                    if len(posCheck) == 2:
                        b1 = int(posCheck[0])
                        b2 = int(posCheck[1])
                        r = [*range(b1, b2+1, 1)] # include 22,24 is 22,23,24 range
                        if pos in r:
                            # check exact match 
                            flounder = True
                            break      
                    if len(posCheck) == 1:
                        try:
                            b1 = int(posCheck[0])
                        except:
                            print("error posCheck:", posCheck)
                            b1=-100
                        if pos == b1:
                            # check exact match 
                            flounder = True
                            break     
                # PROXIMAL
                for prox in vsplit:
                    posCheck = prox.strip(" ")
                    posCheck = posCheck.split("-")
                    try:                            
                        b1 = int(posCheck[0])
                        b2 = int(posCheck[1])
                    except:
                        print("error posCheck:", posCheck)
                        b1=-100
                        b2=-100
                    # check proximal match, within |5 aa| from lower and upper bound
                    if (pos >= b1-5 and pos < b1) | (pos <= b2+5 and pos > b2):
                        flounder2 = True
                        break
                newrow.append(v)
                newrow.append(flounder)
                newrow.append(flounder2)                    
        with open(outfile, 'a') as out:
            csvWriter = csv.writer(out)
            csvWriter.writerow(newrow)
            
#addRegionAnnotations(saveRegion, newheaderRegion, outRegion, regionCol)
```

In [136]:
# qc
l= ['22-24']
for pp in l:
    pc = pp.strip(' ')
    pc = pc.split('-')
    print(pc)
    b1 = int(pc[0])
    b2 = int(pc[1])
    r = [*range(b1, b2+1, 1)] # inclusive
    print(r)

['22', '24']
[22, 23, 24]
